In [1]:
pip install matplotlib numpy scipy seaborn pandas statsmodels jupyterlab plotly notebook 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))


import Models.linear_model 
from Filters.ground_truth import MonteCarloMHFilter
from Filters.linear_kalman  import LinearKalmanFilter
from Filters.ukf  import UnscentedKalmanFilter
from Filters.particle import ParticleFilter

In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [3]:
np.random.seed(123)

d = 2   # dimensión estado
m = 1   # dimensión observación
T = 50

A = np.array([[1.0, 0.1],
              [0.0, 1.0]])

H = np.array([[1.0, 0.0]])

Sigma = 0.01 * np.eye(d)
Gamma = 0.1 * np.eye(m)


In [4]:
def simulate_lgssm(A, H, Sigma, Gamma, x0, T):
    d = len(x0)
    m = H.shape[0]

    X = np.zeros((T + 1, d))
    Y = np.zeros((T, m))

    X[0] = x0
    for t in range(T):
        X[t + 1] = A @ X[t] + np.random.multivariate_normal(np.zeros(d), Sigma)
        Y[t] = H @ X[t + 1] + np.random.multivariate_normal(np.zeros(m), Gamma)

    return X, Y

x0 = np.array([0.0, 1.0])
X_true, Y = simulate_lgssm(A, H, Sigma, Gamma, x0, T)


In [5]:

def A_fn(theta): return A
def H_fn(theta): return H
def Sigma_fn(theta): return Sigma
def Gamma_fn(theta): return Gamma

def Phi(x, theta):
    return A @ x

def h(x, theta):
    return H @ x

def prior_logpdf(theta):
    return 0.0  # fijo para benchmarking

m0 = np.zeros(d)
P0 = np.eye(d)
theta = None


In [6]:
t0 = time.time()

kf = LinearKalmanFilter(
    theta, m0, P0, prior_logpdf,
    A_fn, H_fn, Sigma_fn, Gamma_fn
)


kf_res = kf.filter(Y)
kf_time = time.time() - t0


In [7]:
kf_res

{'log_prior': 0.0,
 'log_likelihood': np.float64(-29.205759990302337),
 'log_posterior': np.float64(-29.205759990302337),
 'prediction': (array([7.64087785, 2.22346979]),
  array([[0.03316331, 0.025858  ],
         [0.025858  , 0.1282876 ]]))}

In [8]:


t0 = time.time()

ukf = UnscentedKalmanFilter(
    theta, m0, P0, prior_logpdf,
    Phi, h, Sigma_fn, Gamma_fn
)

ukf_res = ukf.filter(Y)

ukf_time = time.time() - t0

In [9]:
ukf_res

{'log_prior': 0.0,
 'log_likelihood': np.float64(-29.20576322913742),
 'log_posterior': np.float64(-29.20576322913742),
 'prediction': (array([7.64087778, 2.22346973]),
  array([[0.03316329, 0.02585799],
         [0.02585799, 0.12828748]]))}

In [10]:
t0 = time.time()

pf = ParticleFilter(
    theta, N=1000,
    m0=m0, P0=P0,
    prior_logpdf=prior_logpdf,
    Phi=Phi, h=h,
    Sigma_fn=Sigma_fn,
    Gamma_fn=Gamma_fn
)

pf_res = pf.filter(Y)

pf_time = time.time() - t0

In [11]:
pf_res

{'log_prior': 0.0,
 'log_likelihood': np.float64(-30.1676062633519),
 'log_posterior': np.float64(-30.1676062633519),
 'prediction': (array([7.61214091, 2.09820485]),
  array([[0.03491289, 0.03209801],
         [0.03209801, 0.12612054]])),
 'particles': array([[7.55021214, 1.8417186 ],
        [7.42558065, 1.89413614],
        [7.73389101, 1.89191024],
        ...,
        [7.74374336, 2.58686988],
        [7.84814473, 2.21624688],
        [7.50439278, 1.69682038]], shape=(1000, 2)),
 'weights': array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
   

In [12]:
def rmse(x_hat, x_true):
    return np.sqrt(np.mean((x_hat - x_true)**2))

kf_rmse = rmse(kf_res["prediction"][0], X_true[-1])
ukf_rmse = rmse(ukf_res["prediction"][0], X_true[-1])
pf_rmse = rmse(pf_res["prediction"][0], X_true[-1])

kf_ll = kf_res["log_likelihood"]
ukf_ll = ukf_res["log_likelihood"]
pf_ll = pf_res["log_likelihood"]

print("\n=== COMPARACIÓN DE FILTROS (MODELO LINEAL) ===\n")

print(f"KF  | RMSE: {kf_rmse:.5f} | logL: {kf_ll:.5f} | time: {kf_time:.3f}s")
print(f"UKF | RMSE: {ukf_rmse:.5f} | logL: {ukf_ll:.5f} | time: {ukf_time:.3f}s")
print(f"PF  | RMSE: {pf_rmse:.5f} | logL: {pf_ll:.5f} | time: {pf_time:.3f}s")



=== COMPARACIÓN DE FILTROS (MODELO LINEAL) ===

KF  | RMSE: 0.15312 | logL: -29.20576 | time: 0.010s
UKF | RMSE: 0.15312 | logL: -29.20576 | time: 0.028s
PF  | RMSE: 0.09389 | logL: -30.16761 | time: 3.077s


In [16]:
def prior_sampler():
    return np.random.multivariate_normal(m0, P0)

def transition_sampler(x, theta):
    return A @ x + np.random.multivariate_normal(np.zeros(d), Sigma)

def log_likelihood(y, x, theta):
    diff = y - H @ x
    return -0.5 * (
        diff.T @ np.linalg.inv(Gamma) @ diff
        + np.log(np.linalg.det(Gamma))
        + len(y) * np.log(2 * np.pi)
    )

def proposal_sampler(x):
    return x + 0.2 * np.random.randn(len(x))

def proposal_logpdf(x_new, x_old):
    diff = x_new - x_old
    return -0.5 * diff @ diff / 0.04

mhf = MonteCarloMHFilter(
    theta=None,
    N_pred=500,
    N_mcmc=500,
    prior_sampler=prior_sampler,
    transition_sampler=transition_sampler,
    log_likelihood=log_likelihood,
    proposal_sampler=proposal_sampler,
    proposal_logpdf=proposal_logpdf,
    burn_in=500,
    thinning=5
)

mhf.initialize(N_init=500)

res_mh = mhf.filter(Y)

res_mh

{'log_likelihood': np.float64(-316.2514027825347),
 'prediction': (array([ 7.81597255, 15.52494235]),
  array([[0.08860657, 0.07040702],
         [0.07040702, 8.67028944]])),
 'particles': array([[ 7.73812595, 21.67929613],
        [ 7.69179555, 21.77171062],
        [ 7.82420067, 20.7928494 ],
        [ 7.99311703, 21.0387857 ],
        [ 8.12785568, 21.37705099],
        [ 7.78952314, 21.02506442],
        [ 7.67586911, 20.64096338],
        [ 8.17172839, 20.42473636],
        [ 7.93810209, 19.80733884],
        [ 7.68611209, 19.17946248],
        [ 7.75449382, 18.89135504],
        [ 8.04082319, 19.47105976],
        [ 8.31111964, 19.65445908],
        [ 8.18651922, 19.75844932],
        [ 8.17822924, 19.89050932],
        [ 7.56893051, 20.00109819],
        [ 7.81856386, 19.93986904],
        [ 7.90230925, 19.90694483],
        [ 7.71960112, 19.30293425],
        [ 7.70035738, 18.76962806],
        [ 8.04273475, 19.30731939],
        [ 7.45481643, 19.28203203],
        [ 7.4712826 